#### Recommender system based on past user ratings.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import difflib
import warnings
import re
pd.set_option('display.max_columns', None)
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

##  Loading Dataframes

In [2]:
df_links= pd.read_csv('data/links_small.csv')
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [3]:
df_metadata= pd.read_csv('data/movies_metadata.csv', low_memory=False)
df_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
df_rating= pd.read_csv('data/ratings_small.csv')
df_rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
print(df_links.shape)
print(df_metadata.shape)
print(df_rating.shape)

(9125, 3)
(45466, 24)
(100004, 4)


## Row Filtering

In [6]:
#change all data to integer in id column of df_metadata
# while missing and empty rows are replaced with null value

i = 0
for x in df_metadata['id']:
    if x.isdigit()==False:
        df_metadata.loc[i, 'id']= np.nan
    else:
        df_metadata.loc[i, 'id']= int(df_metadata.loc[i, 'id'])
    i=i+1

In [7]:
ids= df_links['tmdbId'].tolist()

In [8]:
df_metadata1= df_metadata[df_metadata['id'].isin(ids)]
df_metadata1.shape

(9102, 24)

In [9]:
# to ensure that same movies are used in df_metadata and df_rating
df_rating= df_rating[df_rating['movieId'].isin(df_links['movieId'].tolist())]
df_rating.shape

(100004, 4)

## User and Movie Selection

In [10]:
# selecting only users that have given up to 20 reviews 
count =df_rating['userId'].value_counts()
needed_id=count[count>=20].index
df_rates= df_rating[df_rating['userId'].isin(needed_id)]

# selecting only movies with up to 50 reviews
count_m =df_rating['movieId'].value_counts()
needed_movie=count_m[count_m>=50].index
df_rate=df_rates[df_rates['movieId'].isin(needed_movie)]

In [11]:
print(df_rates.shape)
print(df_rate.shape)
df_rate.head()

(100004, 4)
(43083, 4)


,userId,movieId,rating,timestamp
8,1,1339,3.5,1260759125
16,1,2294,2.0,1260759108
19,1,3671,3.0,1260759117
20,2,10,4.0,835355493
21,2,17,5.0,835355681


In [12]:
# merging df_ratings dataframe with df_metadata
df_ratings=pd.merge(df_links, df_rate, on ='movieId')
df_ratings.rename(columns={'tmdbId':'id'}, inplace=True)
df= pd.merge(df_ratings, df_metadata1, on='id')

## Feature Selection

In [13]:
df1=df[['title', 'userId', 'rating']]
df1.head()


,title,userId,rating
0,Toy Story,7,3.0
1,Toy Story,9,4.0
2,Toy Story,13,5.0
3,Toy Story,15,2.0
4,Toy Story,19,3.0


## Data Analysis

In [14]:
df1.isnull().sum()

title     0
userId    0
rating    0
dtype: int64

In [15]:
df1.duplicated().sum()

0

## Matrix Formation and Similarity

In [16]:

df_matrix= df1.pivot_table(values= 'rating', index='title', columns= 'userId', fill_value=0)
df_matrix

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,0,3.5,0,5.0,0.0,0.0,0,0,0.0,0,0.0,0.0,0,0,0.0,0,0,0.0,0,0.0,0,0,0,0.0,0,0,0.0,0,0.0,0.0,0.0,0,0,2.5,0,0,0.0,0,0,0,0.0,0,0.0,0,2,0,0,0.0,0.0,4.0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0.0,3.5,4,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0,0.0,0.0,5,0.0,0.0,5.0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,4.5,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.0,4.5,0,0,5,0,0.0,0,0.0,0.0,3.0,0,0.0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,4.0,2.5,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0,0,4.0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0.0,4.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0.5,0.0,0,0.0,0,4.0,1.0,0,0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,4.0,

In [17]:
similarity=cosine_similarity(df_matrix)
similarity.shape

(453, 453)

In [18]:
def regex_clean(title):
    clean = re.sub('[^a-z0-9A-Z ]','' ,title)
    clean= clean.lower() #set all alphabets to lower case
    return clean

In [20]:
name= pd.Series(df_matrix.index).apply(regex_clean)
df_similarity=pd.DataFrame(similarity, index=name, columns= name)
df_similarity.to_csv('../cleaned_data/df_similarity.csv') # saving dataframe as csv file

df_similarity.head()

title,10 things i hate about you,12 angry men,2001 a space odyssey,28 days later,300,a beautiful mind,a bugs life,a christmas story,a clockwork orange,a close shave,a few good men,a fish called wanda,a league of their own,ai artificial intelligence,about a boy,ace ventura pet detective,ace ventura when nature calls,addams family values,air force one,airplane,aladdin,alien,aliens,alien,almost famous,amadeus,american beauty,american history x,american pie,american psycho,amlie,analyze this,animal house,annie hall,antz,apocalypse now,apollo 13,arachnophobia,armageddon,army of darkness,as good as it gets,austin powers international man of mystery,austin powers the spy who shagged me,avatar,babe,back to the future,back to the future part ii,back to the future part iii,basic instinct,batman,batman begins,batman forever,batman returns,beauty and the beast,beetlejuice,being john malkovich,best in show,beverly hills cop,beverly hills cop iii,big,big fish,blade,blade runner,blazing saddles,bonnie and clyde,boogie nights,bowfinger,bowling for columbine,braveheart,brazil,bridget joness diary,broken arrow,bruce almighty,butch cassidy and the sundance kid,caddyshack,casablanca,casino,casino royale,casper,cast away,catch me if you can,charlies angels,chasing amy,chicago,chicken run,chinatown,cinderella,citizen kane,city slickers ii the legend of curlys gold,city of god,clear and present danger,clerks,cliffhanger,close encounters of the third kind,clueless,con air,coneheads,congo,contact,crimson tide,crocodile dundee,crouching tiger hidden dragon,cruel intentions,dances with wolves,dark city,das boot,dave,dead man walking,dead poets society,demolition man,desperado,die hard,die hard 2,die hard with a vengeance,dirty dancing,disclosure,district 9,dogma,donnie darko,dr strangelove or how i learned to stop worrying and love the bomb,dracula,dragonheart,dumb and dumber,et the extraterrestrial,ed wood,edward scissorhands,election,enemy of the state,eraser,erin brockovich,eternal sunshine of the spotless mind,executive decision,eyes wide shut,faceoff,fantasia,fargo,fast times at ridgemont high,father of the bride part ii,ferris buellers day off,field of dreams,fight club,finding nemo,forrest gump,four weddings and a funeral,french kiss,full metal jacket,galaxy quest,garden state,gattaca,get shorty,ghost,ghostbusters,gladiator,glory,go,goldeneye,goldfinger,gone in sixty seconds,gone with the wind,good morning vietnam,good will hunting,goodfellas,grease,grosse pointe blank,groundhog day,grumpier old men,happy gilmore,harry potter and the chamber of secrets,harry potter and the goblet of fire,harry potter and the philosophers stone,harry potter and the prisoner of azkaban,heat,heathers,high fidelity,home alone,honey i shrunk the kids,hoop dreams,hot shots part deux,i robot,ice age,in the line of fire,inception,independence day,indiana jones and the last crusade,indiana jones and the temple of doom,inglourious basterds,interview with the vampire,iron man,its a wonderful life,jackie brown,james and the giant peach,jaws,jerry maguire,judge dredd,jumanji,juno,jurassic park,kill bill vol 1,kill bill vol 2,kingpin,la confidential,lady and the tramp,last action hero,lawrence of arabia,leaving las vegas,legends of the fall,leon the professional,lethal weapon,lethal weapon 2,liar liar,life is beautiful,life of brian,like water for chocolate,little miss sunshine,little women,lock stock and two smoking barrels,lost in translation,love actually,mash,magnolia,manhattan,mars attacks,mary poppins,maverick,meet the parents,memento,men in black,mighty aphrodite,million dollar baby,minority report,miss congeniality,mission impossible,mission impossible ii,monsters inc,monty python and the holy grail,moulin rouge,mr hollands opus,mrs doubtfire,much ado about nothing,muriels wedding,my best friends wedding,my big fat greek wedding,my cousin vinny,my fair lady,natural born killers,north by northwest,notting hill,o brother where art thou,oceans eleven,office space,one flew

In [21]:
df_title=pd.DataFrame(name)
df_title=df_title.rename(columns={'title':'clean_title'})
df_title['title']=df_matrix.index
movie_title=df_title['title'].tolist()

#getting a list of the homepages and posters of the titles
url=[]
id=[]
for i in df_title['title']:
    set_df=df_metadata[df_metadata['title']==i]
    page =set_df['homepage'].values[0]
    id_values =set_df['id'].values[0]
    url.append(page)
    id.append(id_values)

#forming new columns for homepage and poster path in df_title
df_title['homepage']=url
df_title['id']=id


df_title.head(10)

,clean_title,title,homepage,id
0,10 things i hate about you,10 Things I Hate About You,NaN,4951
1,12 angry men,12 Angry Men,NaN,389
2,2001 a space odyssey,2001: A Space Odyssey,NaN,62
3,28 days later,28 Days Later,NaN,170
4,300,300,http://300themovie.warnerbros.com,1271
5,a beautiful mind,A Beautiful Mind,http://www.abeautifulmind.com/,453
6,a bugs life,A Bug's Life,http://movies.disney.com/a-bugs-life,9487
7,a christmas story,A Christmas Story,NaN,850
8,a clockwork orange,A Clockwork Orange,NaN,185
9,a close shave,A Close Shave,http://www.wallaceandgromit.com/films/a-close-...,532


In [22]:
# handling missing values

df_title.fillna("Not available", inplace=True)


df_title.to_csv('../cleaned_data/df_title.csv') #saving as csv file

df_title.head(10)

,clean_title,title,homepage,id
0,10 things i hate about you,10 Things I Hate About You,Not available,4951
1,12 angry men,12 Angry Men,Not available,389
2,2001 a space odyssey,2001: A Space Odyssey,Not available,62
3,28 days later,28 Days Later,Not available,170
4,300,300,http://300themovie.warnerbros.com,1271
5,a beautiful mind,A Beautiful Mind,http://www.abeautifulmind.com/,453
6,a bugs life,A Bug's Life,http://movies.disney.com/a-bugs-life,9487
7,a christmas story,A Christmas Story,Not available,850
8,a clockwork orange,A Clockwork Orange,Not available,185
9,a close shave,A Close Shave,http://www.wallaceandgromit.com/films/a-close-...,532


## Recommendation

In [25]:
def colaboratory_recommender(title:str):
    title = regex_clean(title)
    title = difflib.get_close_matches(title ,df_title['clean_title'], cutoff= 0.4, n = 1)[0]    
    print("Movie recommendations for: '{}'".format(title))
    recommended_movies=df_similarity[title].sort_values(ascending=False).iloc[1:6]
    recommended=recommended_movies.index.tolist()
    recommended_movies=df_title[df_title['clean_title'].isin(recommended)]['title'].tolist()
    recommended_homepage=df_title[df_title['title'].isin(recommended_movies)]['homepage'].tolist()
    recommendation=[]
    for i in range(0, len(recommended_movies)):
        recommendation.append((recommended_movies[i], recommended_homepage[i]))
    
    return recommendation

In [26]:
print("Type in Movie name")
title= input()

colaboratory_recommender(title)

Type in Movie name
pirates of the caribean
Movie recommendations for: 'pirates of the caribbean dead mans chest'


[('300', 'http://300themovie.warnerbros.com'),
 ('Batman Begins', 'http://www2.warnerbros.com/batmanbegins/index.html'),
 ('Iron Man', 'http://www.ironmanmovie.com/'),
 ('Pirates of the Caribbean: The Curse of the Black Pearl',
  'http://disney.go.com/disneyvideos/liveaction/pirates/main_site/main.html'),
 ('V for Vendetta', 'http://vforvendetta.warnerbros.com/')]